Loading and Exploring Data

In [1]:
import os 
import pandas as pd
base_path = r"C:/Police forces"
print("Base path:", base_path)

Base path: C:/Police forces


In [ ]:
#verify the folder exists and list contents
print("Exists?", os.path.exists(base_path))
print("Contents:", os.listdir(base_path))

Exists? True
Contents: ['Dyfed-Powys Police', 'Gwent Police', 'North Wales Police', 'South Wales Police']


Load all CSVs

In [6]:
#load all CSVs under C:\Police forces\{force}\*.csv
all_data = []

for force_folder in os.listdir(base_path):
    force_path = os.path.join(base_path, force_folder)
    
    # Skip if not a directory
    if not os.path.isdir(force_path):
        continue
    
    # Walk through all subfolders
    for root, dirs, files in os.walk(force_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                print(f"Loading: {file_path}")
                
                try:
                    df_temp = pd.read_csv(file_path)
                    df_temp["PoliceForce"] = force_folder   # Label each row
                    all_data.append(df_temp)
                except Exception as e:
                    print("Error reading file:", file_path, e)

# Combine all CSVs into one DataFrame
if all_data:
    df = pd.concat(all_data, ignore_index=True)
    print("✅ Data loaded successfully!")
    print("Rows:", len(df))
    print("Columns:", len(df.columns))
else:
    print("❌ No CSV files found — check folder structure.")

Loading: C:/Police forces\Dyfed-Powys Police\2023-08\2023-08-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2023-09\2023-09-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2023-10\2023-10-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2023-11\2023-11-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2023-12\2023-12-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-01\2024-01-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-02\2024-02-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-03\2024-03-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-04\2024-04-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-05\2024-05-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-06\2024-06-dyfed-powys-street.csv
Loading: C:/Police forces\Dyfed-Powys Police\2024-07\2024-07-dyfed-powys-str

Initial Exploration

In [13]:
df = pd.concat(all_data, ignore_index=True)


In [ ]:
#  shape
df.shape

(646436, 13)

In [16]:
# Columns in the dataset
df.columns.tolist()

['Crime ID',
 'Month',
 'Reported by',
 'Falls within',
 'Longitude',
 'Latitude',
 'Location',
 'LSOA code',
 'LSOA name',
 'Crime type',
 'Last outcome category',
 'Context',
 'PoliceForce']

In [17]:
# Data types
df.dtypes

Crime ID                  object
Month                     object
Reported by               object
Falls within              object
Longitude                float64
Latitude                 float64
Location                  object
LSOA code                 object
LSOA name                 object
Crime type                object
Last outcome category     object
Context                  float64
PoliceForce               object
dtype: object

In [18]:
# Missing values
df.isnull().sum()

Crime ID                 102635
Month                         0
Reported by                   0
Falls within                  0
Longitude                 14653
Latitude                  14653
Location                      0
LSOA code                 14653
LSOA name                 14653
Crime type                    0
Last outcome category    102635
Context                  646436
PoliceForce                   0
dtype: int64

In [19]:
# Rows per police force
df['PoliceForce'].value_counts()

PoliceForce
South Wales Police    280967
Gwent Police          147682
North Wales Police    137459
Dyfed-Powys Police     80328
Name: count, dtype: int64

In [20]:
df.drop(columns=['Context'], inplace=True)

In [21]:
df = df.dropna(subset=['Latitude', 'Longitude'])

In [22]:
# Fill missing Crime IDs with placeholder if needed
df['Crime ID'] = df['Crime ID'].fillna('Unknown')

# Fill missing last outcome category
df['Last outcome category'] = df['Last outcome category'].fillna('Unknown')

In [23]:
df.isnull().sum()
df.shape

(631783, 12)

In [25]:
df.isnull().sum()

Crime ID                 0
Month                    0
Reported by              0
Falls within             0
Longitude                0
Latitude                 0
Location                 0
LSOA code                0
LSOA name                0
Crime type               0
Last outcome category    0
PoliceForce              0
dtype: int64

In [26]:
df.dtypes

Crime ID                  object
Month                     object
Reported by               object
Falls within              object
Longitude                float64
Latitude                 float64
Location                  object
LSOA code                 object
LSOA name                 object
Crime type                object
Last outcome category     object
PoliceForce               object
dtype: object

In [27]:
df['Month'] = pd.to_datetime(df['Month'], errors='coerce')

In [28]:
df.dtypes

Crime ID                         object
Month                    datetime64[ns]
Reported by                      object
Falls within                     object
Longitude                       float64
Latitude                        float64
Location                         object
LSOA code                        object
LSOA name                        object
Crime type                       object
Last outcome category            object
PoliceForce                      object
dtype: object

In [37]:
df.columns.tolist()

['Month',
 'Reported by',
 'Falls within',
 'Longitude',
 'Latitude',
 'Location',
 'LSOA code',
 'LSOA name',
 'Crime type',
 'Last outcome category',
 'PoliceForce']

Checking for Duplicates

In [29]:
df.duplicated().sum()

np.int64(26638)

Check for outliers

In [30]:
print(df['Latitude'].min(), df['Latitude'].max())
print(df['Longitude'].min(), df['Longitude'].max())

50.230495 55.234185
-5.306914 1.710476


In [31]:
df = df[(df['Latitude'] >= 49) & (df['Latitude'] <= 60)]
df = df[(df['Longitude'] >= -8) & (df['Longitude'] <= 2)]

In [38]:
df.head()

,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,PoliceForce
0,2023-08-01,Dyfed-Powys Police,Dyfed-Powys Police,-2.273464,53.599647,On or near Mossfield Close,E01004960,Bury 007E,Violence and sexual offences,Unable to prosecute suspect,Dyfed-Powys Police
1,2023-08-01,Dyfed-Powys Police,Dyfed-Powys Police,-3.276655,51.754946,On or near Hafod-Y-Mynydd,W01001376,Caerphilly 001C,Violence and sexual offences,Unable to prosecute suspect,Dyfed-Powys Police
2,2023-08-01,Dyfed-Powys Police,Dyfed-Powys Police,-3.147151,51.480731,On or near Hinton Street,W01001873,Cardiff 038C,Other theft,Unable to prosecute suspect,Dyfed-Powys Police
3,2023-08-01,Dyfed-Powys Police,Dyfed-Powys Police,-3.169263,51.464111,On or near Adelaide Place,W01001942,Cardiff 049B,Theft from the person,Investigation complete; no suspect identified,Dyfed-Powys Police
4,2023-08-01,Dyfed-Powys Police,Dyfed-Powys Police,-4.263992,51.997480,On or near Maes Cader,W01000685,Carmarthenshire 001A,Public order,Unable to prosecute suspect,Dyfed-Powys Police


In [39]:
df.drop(columns=['Falls within', 'LSOA code'], inplace=True)

In [40]:
df.columns

Index(['Month', 'Reported by', 'Longitude', 'Latitude', 'Location',
       'LSOA name', 'Crime type', 'Last outcome category', 'PoliceForce'],
      dtype='object')

In [41]:
df.head()

,Month,Reported by,Longitude,Latitude,Location,LSOA name,Crime type,Last outcome category,PoliceForce
0,2023-08-01,Dyfed-Powys Police,-2.273464,53.599647,On or near Mossfield Close,Bury 007E,Violence and sexual offences,Unable to prosecute suspect,Dyfed-Powys Police
1,2023-08-01,Dyfed-Powys Police,-3.276655,51.754946,On or near Hafod-Y-Mynydd,Caerphilly 001C,Violence and sexual offences,Unable to prosecute suspect,Dyfed-Powys Police
2,2023-08-01,Dyfed-Powys Police,-3.147151,51.480731,On or near Hinton Street,Cardiff 038C,Other theft,Unable to prosecute suspect,Dyfed-Powys Police
3,2023-08-01,Dyfed-Powys Police,-3.169263,51.464111,On or near Adelaide Place,Cardiff 049B,Theft from the person,Investigation complete; no suspect identified,Dyfed-Powys Police
4,2023-08-01,Dyfed-Powys Police,-4.263992,51.997480,On or near Maes Cader,Carmarthenshire 001A,Public order,Unable to prosecute suspect,Dyfed-Powys Police


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 631783 entries, 0 to 646157
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Month                  631783 non-null  datetime64[ns]
 1   Reported by            631783 non-null  object        
 2   Longitude              631783 non-null  float64       
 3   Latitude               631783 non-null  float64       
 4   Location               631783 non-null  object        
 5   LSOA name              631783 non-null  object        
 6   Crime type             631783 non-null  object        
 7   Last outcome category  631783 non-null  object        
 8   PoliceForce            631783 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 48.2+ MB


In [44]:
df.describe()

,Month,Longitude,Latitude
count,631783,631783.000000,631783.000000
mean,2024-08-04 04:35:38.017641984,-3.474004,51.984864
min,2023-08-01 00:00:00,-5.306914,50.230495
25%,2024-02-01 00:00:00,-3.806786,51.577644
50%,2024-08-01 00:00:00,-3.283419,51.662443
75%,2025-03-01 00:00:00,-3.139517,52.092867
max,2025-08-01 00:00:00,1.710476,55.234185
std,NaN,0.487121,0.656775


In [ ]:
df[df['Month'] > '2024-12-31'].shape


(205693, 9)

In [46]:
df = df[(df['Month'] >= '2023-08-01') & (df['Month'] <= '2025-08-31')]

In [47]:
df.shape

(631783, 9)

In [48]:
print(df.shape)
print(df.isnull().sum())
print(df.dtypes)
df.head()

(631783, 9)
Month                    0
Reported by              0
Longitude                0
Latitude                 0
Location                 0
LSOA name                0
Crime type               0
Last outcome category    0
PoliceForce              0
dtype: int64
Month                    datetime64[ns]
Reported by                      object
Longitude                       float64
Latitude                        float64
Location                         object
LSOA name                        object
Crime type                       object
Last outcome category            object
PoliceForce                      object
dtype: object


,Month,Reported by,Longitude,Latitude,Location,LSOA name,Crime type,Last outcome category,PoliceForce
0,2023-08-01,Dyfed-Powys Police,-2.273464,53.599647,On or near Mossfield Close,Bury 007E,Violence and sexual offences,Unable to prosecute suspect,Dyfed-Powys Police
1,2023-08-01,Dyfed-Powys Police,-3.276655,51.754946,On or near Hafod-Y-Mynydd,Caerphilly 001C,Violence and sexual offences,Unable to prosecute suspect,Dyfed-Powys Police
2,2023-08-01,Dyfed-Powys Police,-3.147151,51.480731,On or near Hinton Street,Cardiff 038C,Other theft,Unable to prosecute suspect,Dyfed-Powys Police
3,2023-08-01,Dyfed-Powys Police,-3.169263,51.464111,On or near Adelaide Place,Cardiff 049B,Theft from the person,Investigation complete; no suspect identified,Dyfed-Powys Police
4,2023-08-01,Dyfed-Powys Police,-4.263992,51.997480,On or near Maes Cader,Carmarthenshire 001A,Public order,Unable to prosecute suspect,Dyfed-Powys Police
